In [23]:
import pandas as pd
import numpy as np

import datetime as dt
import string

import matplotlib.pyplot as plt

In [24]:
from xbbg import blp

### Parameters

In [25]:
WRITE_DATA = True

TODAY = dt.datetime.today().strftime('%Y%m%d')  
OUTFILE = f'../build_data/spx_carry_{TODAY}.xlsx'

# Get SPX Members

#### BB Call

In [26]:
DT_MEMBERS = TODAY
weights = blp.bds(['SPX Index'],flds=['INDX_MWEIGHT_HIST'],END_DATE_OVERRIDE=DT_MEMBERS)

In [27]:
TICKS_SPX = weights.index_member.str.split().str.get(0)
TICKS_SPX_BB = (TICKS_SPX + [' US Equity']).to_list()
TICKS_SPX = TICKS_SPX.to_list()

weights.set_index('index_member',inplace=True)
weights.index = weights.index.str.split(' ').str[0]

spx_members = pd.DataFrame({'tickers':TICKS_SPX, 'BB tickers':TICKS_SPX_BB})
spx_members.set_index('tickers',inplace=True)

tickerBB_to_ticker = spx_members.reset_index().set_index('BB tickers')['tickers'].to_dict()

## Download Ticker Info

#### BB Call

In [28]:
info_spx = blp.bdp(TICKS_SPX_BB,flds=['SECURITY_NAME','EQY_DVD_YLD_IND'])

In [29]:
info_spx = info_spx.rename(index=tickerBB_to_ticker)

# Download Adjusted Prices for all SPX Members

In [30]:
STARTDATE = '2014-12-31'
ENDDATE = '2024-12-31'

FLD = 'EQY_DVD_YLD_IND'

#### BB Call. Careful: Running this takes awhile and uses lots of BB Data Limit

In [31]:
dvd_raw = blp.bdh(TICKS_SPX_BB,flds=[FLD],start_date=STARTDATE,end_date=ENDDATE)

In [32]:
dvd = dvd_raw.droplevel(level=1,axis=1)
dvd.rename(columns=tickerBB_to_ticker,inplace=True)

### Clean data for export

In [33]:
dvd.index.name = 'date'
info_spx.index.name = 'ticker'
cols = dvd.columns
cols = [col.replace(' US Equity','') for col in cols]
dvd.columns = cols   

## Save Data

Takes a full minute or so to export the SPX member prices

In [34]:
if WRITE_DATA:
    with pd.ExcelWriter(OUTFILE) as writer:  
        info_spx.to_excel(writer, sheet_name= 'spx names')
        
        dvd.to_excel(writer, sheet_name='dvd yield')

***

# Process

In [35]:
INFILE = OUTFILE

In [36]:
spx_raw = pd.read_excel(INFILE,sheet_name='dvd yield').set_index('date')

In [37]:
info_spx = pd.read_excel(INFILE,sheet_name='spx names').set_index('ticker')
info_spx.columns = ['security name','dvd yield']

In [38]:
DT0 = '2016'
AGG = 'W-FRI'
FREQ = 52

spx = spx_raw.resample(AGG).last().iloc[:-1,:]
spx /= 100

In [40]:
OUTFILE = '../data/spx_carry_weekly.xlsx'

with pd.ExcelWriter(OUTFILE) as writer:  
    info_spx.to_excel(writer, sheet_name= 's&p500 names')    
    spx.to_excel(writer, sheet_name=f's&p500 dvd yield')